In [12]:
import pandas as pd
import constants as c
import os 

bpa_dir = c.bpa_dir

bpas = dict()

codigos = [f for f in os.listdir(bpa_dir) if os.path.isdir(os.path.join(bpa_dir, f))]
for cod in codigos:
    file = [f for f in os.listdir(f"{bpa_dir}{cod}/") if os.path.isfile(os.path.join(f"{bpa_dir}{cod}/", f))][0]
    path = f"{bpa_dir}{cod}/{file}"
    print(path)
    bpas[int(cod)] = pd.read_csv(path)


./demonstrativos.nosync/bpa/24406/BANCO INTER S.A.-31-12-2021.csv
./demonstrativos.nosync/bpa/21180/Nexpe Participações S.A-31-12-2021.csv
./demonstrativos.nosync/bpa/20052/DINAMICA ENERGIA S.A.-31-12-2021.csv
./demonstrativos.nosync/bpa/12190/BOMBRIL S.A.-31-12-2021.csv
./demonstrativos.nosync/bpa/25712/GPS PARTICIPAÇÕES E EMPREENDIMENTOS S.A.-31-12-2021.csv
./demonstrativos.nosync/bpa/14869/CIA ENERGETICA DO CEARA - COELCE-31-12-2021.csv
./demonstrativos.nosync/bpa/25143/PACAEMBU CONSTRUTORA S.A.-31-12-2021.csv
./demonstrativos.nosync/bpa/26190/CARAMURU ALIMENTOS S.A.-31-12-2020.csv
./demonstrativos.nosync/bpa/24295/VIBRA ENERGIA S_*_A-31-12-2021.csv
./demonstrativos.nosync/bpa/23612/MAESTRO LOCADORA DE VEICULOS S.A.-31-12-2021.csv
./demonstrativos.nosync/bpa/26360/DORI ALIMENTOS S.A.-31-12-2021.csv
./demonstrativos.nosync/bpa/24430/COMPANHIA ENERGÉTICA MIRANDA-31-12-2021.csv
./demonstrativos.nosync/bpa/25186/GRUPO MATEUS S.A.-31-12-2021.csv
./demonstrativos.nosync/bpa/22357/Aliansce

In [31]:
essential_cols = [
        ' Ativo Total ', ' Ativo Circulante ', ' Ativo Não Circulante ', ' Caixa e Equivalentes de Caixa ', ' Contas a Receber ',
        ' Ativo Realizável a Longo Prazo ', ' Imobilizado '
    ]

valid_bpas = dict()

for bpa_cod in bpas:
    bpa = bpas[bpa_cod]
    # print(bpa.head())
    if not ' Descrição ' in bpa.columns: continue

    contas = bpa[' Descrição ']

    # # discards banks and other tickers that do not have ' Receita de Venda de Bens e/ou Serviços '
    # if ' Receitas da Intermediação Financeira ' in contas.values: 
    #     continue
    # if not ' Receita de Venda de Bens e/ou Serviços ' in contas.values:
    #     continue


    invalid = False
    for attr in essential_cols:
        for c in [' 31/12/2019 ', ' 31/12/2020 ', ' 31/12/2021 ']:
            # remove the ones without any of the essential cols
            rec = bpa[bpa[' Descrição '] == attr]
            if not c in rec or len(rec[c].values) == 0:
                invalid = True
                break
            receita = rec[c].values[0].replace(' ', '')
            if receita == '': 
                invalid = True
                break
    if invalid: continue
    
    valid_bpas[bpa_cod] = bpa

print(f'{len(valid_bpas)} valid bpas')

336 valid bpas


In [243]:
market_caps = pd.read_csv('market_caps.csv')
cvm_to_ticker = pd.read_csv('cvm_to_ticker.csv')

market_caps
cvm_to_ticker

mc = dict()

for t in market_caps['Ticker']:
    cvm = cvm_to_ticker[cvm_to_ticker['Ticker'] == t]['CVM'].values[0]
    mc[cvm] = {
        'ticker': t,
        'market_cap': market_caps[market_caps['Ticker'] == t]
    }

valid_keys = mc.keys() & valid_bpas.keys()
valid_bpas = {k: valid_bpas[k] for k in valid_keys}

In [267]:
for bpa_cod in valid_bpas:
    bpa = valid_bpas[bpa_cod]

    
    for year in [2019, 2020, 2021]:
        bpa_fields = dict()
        for e in essential_cols:
            conta_row = bpa[bpa[' Descrição '] == e]
            e = e.replace(' ', '').replace('/', '').lower()
            bpa_fields[e] = conta_row[f'31/12/{year} a 31/12/{year} '].values[0].replace(' ', '')
        print(bpa_fields)


# valid_bpas[list(valid_bpas.keys())[0]]

{'receitadevendadebenseouserviços': '677334', 'resultadobruto': '76134', 'custodosbenseouserviçosvendidos': '-601200', 'despesasreceitasoperacionais': '-60426', 'resultadoantesdoresultadofinanceiroedostributos': '15708', 'resultadofinanceiro': '-20216', 'resultadoantesdostributossobreolucro': '-4508', 'impostoderendaecontribuiçãosocialsobreolucro': '-9377', 'resultadolíquidodasoperaçõescontinuadas': '-13885', 'lucroprejuízoconsolidadodoperíodo': '-13885'}
{'receitadevendadebenseouserviços': '618625', 'resultadobruto': '103689', 'custodosbenseouserviçosvendidos': '-514936', 'despesasreceitasoperacionais': '-102550', 'resultadoantesdoresultadofinanceiroedostributos': '1139', 'resultadofinanceiro': '-46496', 'resultadoantesdostributossobreolucro': '-45357', 'impostoderendaecontribuiçãosocialsobreolucro': '-563', 'resultadolíquidodasoperaçõescontinuadas': '-45920', 'lucroprejuízoconsolidadodoperíodo': '-45920'}
{'receitadevendadebenseouserviços': '954276', 'resultadobruto': '149520', 'cust